In [48]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import os
from PIL import Image
import numpy as np
from torch.utils import data
from torchvision import transforms as T
from torch.utils.data import DataLoader
from scipy import io

transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[128, 128, 128], std=[1, 1, 1])
    # T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

d_label = {}
with open('./list/pList.txt','r') as f:
    for line in f.readlines():
        tmp = line.split('/')[-1].strip('\n').strip('\r').split(' ')
        d_label[tmp[0]] = tmp[1]

In [49]:
class VOC07(data.Dataset):
    def __init__(self, root, transforms=None):
        imgs = os.listdir(root)
        self.imgs = [os.path.join(root, img) for img in imgs]
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.imgs[index]
        data = Image.open(img_path)
        label = d_label[img_path.split('/')[-1]]
        if self.transforms:
            data = self.transforms(data)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

In [50]:
class SppNet(nn.Module):
    def __init__(self, batch_size=1, out_pool_size=[1,2,4], class_number=2):
        super(SppNet, self).__init__()
        vgg = torchvision.models.vgg16(pretrained=True).features
        vgg = nn.Sequential(*list(vgg.children())[:-1])
        self.out_pool_size = out_pool_size
        self.batch_size = batch_size
        self.encoder = vgg
        self.spp = self.make_spp(batch_size=batch_size, out_pool_size=out_pool_size)
        sum0 = 0
        for i in out_pool_size:
            sum0 += i**2
        self.fc = nn.Sequential(nn.Linear(512*sum0, 1024), nn.ReLU(inplace=True))
        self.score = nn.Linear(1024, class_number)
    
    def make_spp(self, batch_size=1, out_pool_size=[1,2,4]):
        func = []
        for i in range(len(out_pool_size)):
            func.append(nn.AdaptiveAvgPool2d(output_size=(out_pool_size[i], out_pool_size[i])))
        return func
    
    def forward(self, x):
        assert x.shape[0] == 1 , 'batch size need to set to be 1'
        encoder = self.encoder(x)
        spp=[]
        for i in range(len(self.out_pool_size)):
            spp.append(self.spp[i](encoder).view(self.batch_size, -1))
        fc = torch.cat(spp, dim=1)
        return fc

In [51]:
dataset = VOC07('./p/', transforms=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=3)
spp = SppNet(out_pool_size=[6], class_number=20)
spp = spp.cuda()
features = torch.zeros((1, 18432))
features = features.cuda()
labels = []
for batch_data, batch_label in dataloader:
    batch_data = Variable(batch_data.cuda())
    out = spp(batch_data)
    out_np = out.data
    print(batch_label[0])
    labels.append(int(batch_label[0]))
    features = torch.cat((features, out_np), 0)
feature = features[1:,:]
feature = feature.cpu().numpy()
feature.shape
label = np.matrix(labels)
io.savemat('p_features.mat', {'p_feature': feature})
io.savemat('p_labels.mat', {'p_label': label})

8
9
4
6
5
11
9
0
8
10
2
3
1
1
3
9
1
5
9
8
8
10
6
6
11
11
2
1
4
2
10
0
6
0
0
9
2
0
9
7
5
9
7
11
7
2
10
10
8
3
6
9
2
4
8
10
8
1
8
7
5
5
7
7
7
5
11
8
7
5
6
6
8
5
3
3
10
7
2
0
4
6
0
6
4
8
1
11
2
1
9
0
2
3
9
0
11
1
9
3
1
4
10
9
8
8
3
1
3
4
11
6
4
8
4
11
8
1
4
10
5
10
7
0
2
9
9
9
11
6
0
3
7
0
11
1
4
10
6
6
5
8
11
6
0
7
9
4
0
9
0
1
3
3
11
10
6
11
4
6
2
10
11
9
3
11
11
3
6
1
2
11
5
1
4
2
0
4
9
6
2
8
10
1
0
5
9
9
0
3
4
0
8
9
4
7
4
8
7
3
0
5
9
3
2
8
7
8
0
1
7
1
4
6
3
6
2
10
1
0
11
9
8
8
0
8
2
9
4
0
2
10
1
8
10
0
10
7
1
10
7
9
0
2
11
4
10
8
7
2
10
10
10
10
8
3
4
2
7
11
4
6
11
2
6
11
2
6
7
8
2
2
8
1
7
1
6
7
9
10
8
8
5
10
8
0
1
11
5
7
0
0
4
0
7
7
3
9
11
8
3
2
0
11
9
3
8
9
7
5
11
0
8
5
6
3
1
0
6
7
9
2
5
2
10
8
8
11
11
2
5
11
0
6
11
5
0
0
2
0
0
10
8
2
2
0
1
11
2
9
10
3
6
9
7
2
6
10
2
9
2
2
11
6
5
1
0
8
1
8
6
3
5
11
9
8
1
9
11
3
3
7
1
5
3
9
8
4
10
8
11
5
2
5
2
4
1
5
11
11
7
9
5
0
10
5
5
1
1
6
8
8
6
9
4
6
11
4
2
5
2
2
6
10
0
1
10
6
9
1
7
4
4
3
6
5
11
3
0
1
6
4
3
7
10
3
1
10
1
6
4
10
3
6
4
10
8
1
7
2
5


In [16]:
rm b_normal_labels.mat

In [35]:
rm i_features.mat i_labels.mat